Preparing Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
# grammyAwards = grammyAwards[['Name', 'Artist', 'Genre', 'GrammyAward']]
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist']).count().reset_index()
grammyAwards = grammyAwards.applymap(lambda s: s.upper() if type(s) == str else s)
print(grammyAwards.shape)
grammyAwards.head(5)

In [ ]:
billboardHot100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
billboardHot100 = billboardHot100[['Artists', 'Name', 'Peak.position', 'Weeks.on.chart']]
billboardHot100['Weeks.on.chart'] = billboardHot100['Weeks.on.chart'].fillna(1)
billboardHot100['Peak.position'] = billboardHot100['Peak.position'].fillna(100)
# billboardHot100.groupby(by=['Artist', 'Name']).min('Peak.position').max('Weeks.on.chart')
billboardHot100 = billboardHot100.groupby(['Artists', 'Name']).agg({'Peak.position':'min', 'Weeks.on.chart':'max'})[['Peak.position','Weeks.on.chart']].reset_index()
billboardHot100 = billboardHot100.applymap(lambda s: s.upper() if type(s) == str else s)
billboardHot100.head(5)